In [ ]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
# custom imports
from multiprocessing import Pool        # Multiprocess Runs

warnings.filterwarnings('ignore')

In [ ]:
########################### Helpers
#################################################################################

def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [ ]:
########## Getting the scales and weights #############
path = '/kaggle/input/m5-weights-and-scales/'

def get_weights_scales_level_12(df, end_test, path):
    # Gets the scale, weight, and scaled weight in a dataframe, 
    # aligned with the 'id' column of df

    # Get the weights and scales for all the levels 
    wdf = pd.read_csv(f'{path}weight_scale_{end_test-27}.csv')
    wdf['scaled_weight'] = wdf.weight/np.sqrt(wdf.scale)

    # For this function, we just want level 12 weights and scales
    wdf = wdf[wdf.Level_id == 'Level12']

    # We make an 'id' column for easy merging, df must have 'id' column
    wdf['id'] = wdf['Agg_Level_1'] + '_' +  wdf['Agg_Level_2'] + '_validation'

    # Taking just he columns we want to use in the merge 
    wdf = wdf[['id', 'scale', 'weight', 'scaled_weight']]

    # Merge with 'id' column of the df
    wdf = pd.merge(df[['id']], wdf, on='id', how='left')
    
    return wdf

In [ ]:
def L12_WRMSSE(preds, actuals, p_horizon, num_products, scale, weight): 
    
    actuals = actuals[-(p_horizon * num_products):]
    preds = preds[-(p_horizon * num_products):]
    diff = actuals - preds

    # For WRMSSE with need square of the daily error and then get the appropriate scale
    res = diff ** 2
    res = res/scale.values

    res = res
    res = res.reshape(p_horizon, num_products)

    res = res.mean(axis=0)
    res = np.sqrt(res)

    res = res * weight
    res = res.sum()
    return res
################### Custom metric #####################
def custom_metric(preds, train_data):
    actuals = train_data.get_label()
    res = L12_WRMSSE(preds, actuals, P_HORIZON, NUM_PRODUCTS, scale, weight)
    return 'L12_WRMSSE', res, False